In [1]:
#this works to write to a mysql database, needs to be updated to iterate through the pages


from sqlalchemy import create_engine, Table, Column, MetaData
from sqlalchemy.dialects.mysql import LONGTEXT
from sqlalchemy.orm import sessionmaker
import requests
import csv
from io import StringIO
import arcturis as a 


# Define database connection using SQLAlchemy
DATABASE_URL = a.DB
engine = create_engine(DATABASE_URL)
metadata = MetaData(bind=engine)


# API request
url = "https://clinicaltrials.gov/api/v2/studies?format=csv&query.term=oncology&pageSize=1000"
response = requests.get(url)

if response.status_code == 200:
    csv_data = StringIO(response.content.decode("utf-8"))
    csv_reader = csv.reader(csv_data)
    headers = next(csv_reader)  # Get headers from the first row

    # Define table dynamically based on headers
    columns = []
    for header in headers:
        header_sanitized = header.strip().replace(" ", "_").replace("-", "_").replace(".", "_")
        columns.append(Column(header_sanitized, LONGTEXT))  # Set all columns as LONGTEXT

    # Create the table with appropriate data types
    clinical_trials = Table('clinical_trials_test', metadata, *columns, extend_existing=True)
    metadata.create_all()

    # Insert data into the database
    Session = sessionmaker(bind=engine)
    session = Session()

    for row in csv_reader:
        if len(row) == len(headers):
            insert_data = dict(zip([col.name for col in clinical_trials.columns], row))
            ins = clinical_trials.insert().values(**insert_data)
            session.execute(ins)
        else:
            print("Skipping row with incorrect number of columns:", row)

    session.commit()

    next_page_token = response.headers.get('x-next-page-token')
    while next_page_token:
        # Make the next API request with the next page token
        response = requests.get(url + f"&pageToken={next_page_token}")
        if response.status_code == 200:
            lines = response.content.decode("utf-8").splitlines()
            csv_reader = csv.reader(lines)
            next(csv_reader)  # Skip the header row
            for row in csv_reader:
                insert_data(cursor, row)
            conn.commit()
            next_page_token = response.headers.get('x-next-page-token')
        else:
            print("Error:", response.status_code)
            break
    session.close()

else:
    print("Error:", response.status_code)



C:\Users\Andre\AppData\Local\Temp\ipykernel_10800\1926518253.py:16: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  metadata = MetaData(bind=engine)


In [3]:
from sqlalchemy import create_engine, Table, Column, MetaData
from sqlalchemy.dialects.mysql import LONGTEXT
from sqlalchemy.orm import sessionmaker
import requests
import csv
csv.field_size_limit(100000000)  # Set the field size limit to a larger value

from sqlalchemy.exc import NoSuchTableError
from io import StringIO
import arcturis as a


host = a.line,
port = 3306,
user = a.linu,
password = a.linp,
database = a.lindb



# Define database connection using SQLAlchemy
DATABASE_URL = a.LDB
engine = create_engine(DATABASE_URL)
metadata = MetaData(bind=engine)


def create_table(metadata):
    try:
        # Try to autoload the existing table
        clinical_trials = Table('clinical_trials_test', metadata, autoload_with=engine)
    except NoSuchTableError:
        # If the table does not exist, create it
        columns = [Column(header.strip().replace(" ", "_").replace("-", "_").replace(".", "_"), LONGTEXT) for header in headers]
        clinical_trials = Table('clinical_trials_test', metadata, *columns, extend_existing=True)
        clinical_trials.create()

# Call the create_table function before using the table
create_table(metadata)


def insert_data(engine, data):
    with engine.connect() as conn:
        clinical_trials = Table('clinical_trials_test', metadata, autoload_with=engine, extend_existing=True)
        ins = clinical_trials.insert().values(**data)
        conn.execute(ins)

# API request
url = "https://clinicaltrials.gov/api/v2/studies?format=csv&query.term=oncology&pageSize=1000"
response = requests.get(url)

if response.status_code == 200:
    csv_data = StringIO(response.content.decode("utf-8"))
    csv_reader = csv.reader(csv_data)
    headers = next(csv_reader)  # Get headers from the first row
    create_table(metadata)

    # Insert data into the database
    Session = sessionmaker(bind=engine)
    session = Session()

    for row in csv_reader:
        if len(row) == len(headers):
            insert_data(engine, dict(zip([col.strip().replace(" ", "_").replace("-", "_").replace(".", "_") for col in headers], row)))
        else:
            print("Skipping row with incorrect number of columns:", row)

    session.commit()
    
    next_page_token = response.headers.get('x-next-page-token')
    while next_page_token:
        # Make the next API request with the next page token
        response = requests.get(url + f"&pageToken={next_page_token}")
        if response.status_code == 200:
            lines = response.content.decode("utf-8").splitlines()
            csv_reader = csv.reader(lines)
            next(csv_reader)  # Skip the header row
            for row in csv_reader:
                insert_data(engine, dict(zip([col.strip().replace(" ", "_").replace("-", "_").replace(".", "_") for col in headers], row)))
            session.commit()
            next_page_token = response.headers.get('x-next-page-token')
        else:
            print("Error:", response.status_code)
            break
    session.close()

else:
    print("Error:", response.status_code)


Exception during reset or similar
Traceback (most recent call last):
  File "c:\Users\Andre\AppData\Local\Programs\Python\Python311\Lib\site-packages\pymysql\connections.py", line 779, in _read_bytes
    data = self._rfile.read(num_bytes)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Andre\AppData\Local\Programs\Python\Python311\Lib\socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\Andre\AppData\Local\Programs\Python\Python311\Lib\site-packages\sqlalchemy\pool\base.py", line 763, in _finalize_fairy
    fairy._reset(pool, transaction_was_reset)
  File "c:\Users\Andre\AppData\Local\Programs\Python\Python311\Lib\site-packages\sqlalchemy\pool\base.py", line 1038, in _reset
    pool._dialect.do_rollback(self)

OperationalError: (pymysql.err.OperationalError) (2013, 'Lost connection to MySQL server during query ([WinError 10054] An existing connection was forcibly closed by the remote host)')
(Background on this error at: https://sqlalche.me/e/14/e3q8)